In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.stats import binomtest, chi2

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
df = pd.read_excel("df2.xlsx", usecols="A:B", skiprows=1)
criteria = pd.read_excel("df2.xlsx", usecols="D:F", skiprows=1)

In [ ]:
criteria.head(5)

In [ ]:
df.head(5)

In [5]:
df.rename(columns={"Điểm": "Score", "Good=0/Bad=1": "Label"}, inplace=True)
criteria.rename(columns={"Master Scale": "Scale", "Điểm.1": "Criteria"}, inplace=True)

In [ ]:
X = df[['Score']]
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42)
}
param_grids = {
    "Random Forest": {'n_estimators': [50, 100], 'max_depth': [5, None]},
    "Gradient Boosting": {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1]},
    "Decision Tree": {'max_depth': [5, None]},
    "Logistic Regression": {'C': [0.01, 0.1, 1.0]}
}

best_models = {}
for name, model in models.items():
    grid = GridSearchCV(model, param_grids[name], cv=5, scoring='roc_auc')
    grid.fit(X_train, y_train)
    best_models[name] = (grid.best_estimator_, grid.best_params_, grid.best_score_)
    gini = 2 * grid.best_score_ - 1
    print(f"{name}: {grid.best_params_}, ROC AUC: {grid.best_score_:.4f}, Gini: {gini:.4f}")

In [ ]:
best_models

In [ ]:
param_boost = {
    "XGBoost": {"n_estimators": [50, 100], "max_depth": [3, 5], "learning_rate": [0.01, 0.1]},
    "LightGBM": {"n_estimators": [20, 50], "max_depth": [3, 5], "learning_rate": [0.01, 0.1]}
}

models.update({
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
})

for name in ["XGBoost", "LightGBM"]:
    grid = GridSearchCV(models[name], param_boost[name], cv=5, scoring='roc_auc')
    grid.fit(X_train, y_train)
    best_models[name] = (grid.best_estimator_, grid.best_params_, grid.best_score_)
    print(f"{name} Best Params: {grid.best_params_}, ROC AUC: {grid.best_score_:.4f}, Gini: {2*grid.best_score_-1:.4f}")


In [ ]:
for name, (estimator, _, _) in best_models.items():
    y_proba = estimator.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.plot(fpr, tpr, label=name)

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves by Model')
plt.legend()
plt.show()

In [ ]:
results = pd.DataFrame(best_models)

summary_df = pd.DataFrame(
    [[k, v[2], 2*v[2] - 1] for k, v in best_models.items()],
    columns=['model_name', 'ROC', 'gini']
)
print(summary_df)


In [28]:
def hosmer_lemeshow_test(y_true, y_pred, n_groups=10):
    """
    Perform Hosmer-Lemeshow test
    
    Parameters:
    y_true (array): Actual binary outcomes
    y_pred (array): Predicted probabilities
    n_groups (int): Number of groups for the test
    """
    # Sort predictions and create groups
    indices = np.argsort(y_pred)
    y_true = y_true[indices]
    y_pred = y_pred[indices]
    
    # Create groups
    size = len(y_true) // n_groups
    groups = [y_true[i:i + size] for i in range(0, len(y_true), size)]
    pred_groups = [y_pred[i:i + size] for i in range(0, len(y_pred), size)]
    
    # Calculate observed and expected frequencies
    observed = np.array([sum(group) for group in groups])
    expected = np.array([sum(pred_group) for pred_group in pred_groups])
    
    # Calculate chi-square statistic
    chi_square = np.sum((observed - expected) ** 2 / (expected * (1 - expected/size)))
    
    # Calculate p-value (df = n_groups - 2)
    p_value = 1 - chi2.cdf(chi_square, n_groups - 2)
    return chi_square, p_value

In [31]:
from scipy.stats import binomtest
import numpy as np

for model in best_models:
    y_proba = best_models[model][0].predict_proba(X_test)[:, 1]  # Predicted probabilities
    mean_predicted_prob = np.mean(y_proba)  # Average predicted probability
    expected_successes = sum(y_proba)  # Sum of predicted probabilities

    # Binomial test
    observed_successes = sum(y_test)
    num_trials = len(y_test)
    binom_pvalue = binomtest(observed_successes, num_trials, mean_predicted_prob).pvalue

    print(f"{model}:")
    print(f"Observed Successes: {observed_successes}")
    print(f"Expected Successes (Model): {expected_successes:.2f}")
    print(f"Binomial Test p-value: {binom_pvalue:.4f}")


Random Forest:
Observed Successes: 139
Expected Successes (Model): 133.16
Binomial Test p-value: 0.5794
Gradient Boosting:
Observed Successes: 139
Expected Successes (Model): 134.24
Binomial Test p-value: 0.6454
Decision Tree:
Observed Successes: 139
Expected Successes (Model): 133.63
Binomial Test p-value: 0.6119
Logistic Regression:
Observed Successes: 139
Expected Successes (Model): 135.23
Binomial Test p-value: 0.7136
XGBoost:
Observed Successes: 139
Expected Successes (Model): 131.66
Binomial Test p-value: 0.4863
LightGBM:
Observed Successes: 139
Expected Successes (Model): 132.36
Binomial Test p-value: 0.5470


In [34]:
for model in best_models:
    y_proba = best_models[model][0].predict_proba(X_test)[:, 1]  
    mean_predicted_prob = np.mean(y_proba)  # Average predicted probability
    expected_successes = sum(y_proba)  # Sum of predicted probabilities
    observed_successes = sum(y_test)
    num_trials = len(y_test)
    
    print(f"{model}:")
    print(f'Binomial Test 2 tails : {binomtest(observed_successes, num_trials, mean_predicted_prob, alternative="two-sided").pvalue:.4f}')
    print(f'Binomial Test 1 tail : {binomtest(observed_successes, num_trials, mean_predicted_prob, alternative="greater").pvalue:.4f}')
    print(f"Hosmer-Lemeshow Test: {hosmer_lemeshow_test(y_test.values, y_proba)}")
    print("\n")


Random Forest:
Binomial Test 2 tails : 0.5794
Binomial Test 1 tail : 0.3079
Hosmer-Lemeshow Test: (156.74741107520205, 0.0)


Gradient Boosting:
Binomial Test 2 tails : 0.6454
Binomial Test 1 tail : 0.3438
Hosmer-Lemeshow Test: (10.378005419429426, 0.23949052509886692)


Decision Tree:
Binomial Test 2 tails : 0.6119
Binomial Test 1 tail : 0.3235
Hosmer-Lemeshow Test: (nan, nan)


Logistic Regression:
Binomial Test 2 tails : 0.7136
Binomial Test 1 tail : 0.3781
Hosmer-Lemeshow Test: (14.59487953989061, 0.06751826810686656)


XGBoost:
Binomial Test 2 tails : 0.4863
Binomial Test 1 tail : 0.2604
Hosmer-Lemeshow Test: (13.552937103869457, 0.09418822527710602)


LightGBM:
Binomial Test 2 tails : 0.5470
Binomial Test 1 tail : 0.2821
Hosmer-Lemeshow Test: (10.323475819071094, 0.24305209490981683)




/var/folders/nk/6_ymhlt94l97ghl62d13wtym0000gn/T/ipykernel_54758/669386921.py:25: RuntimeWarning: divide by zero encountered in divide
  chi_square = np.sum((observed - expected) ** 2 / (expected * (1 - expected/size)))
/var/folders/nk/6_ymhlt94l97ghl62d13wtym0000gn/T/ipykernel_54758/669386921.py:25: RuntimeWarning: invalid value encountered in divide
  chi_square = np.sum((observed - expected) ** 2 / (expected * (1 - expected/size)))


# Criterias for classification model

## AUC, ROC
TPR = TP / (TP + FN) tỉ lệ dương thật
FPR = FP / (FP + TN) tỉ lê lỗi dương giả

ROC tạo ra bằng cách thay đổi threshold của mô hình đối với prediction, khi giảm threshold, TPR tăng, FPR cũng tăng, và ngược lại

Đọc ROC:
- Đường cong của mô hình là hiệu suất dự đoán của mô hình, càng gần trục Y (TPR) thì mô hình dự đoán càng tốt
- Đường chéo là việc dự đoán ngẫu nhiên ban đầu

AUC: diện tích dưới đường ROC, càng lớn thì mô hình càng tốt, được tính từ TPR và FPR. AUC = 1 thì mô hình hoàn hảo, AUC = 0.5 thì mô hình dự đoán ngẫu nhiên

## Gini
Gini đo lường khả năng phân biệt giữa 2 lớp, là việc chuyển đổi AUC để dễ dàng so sánh
Gini = 2 * AUC - 1, càng lớn thì mô hình càng tốt, Gini = 1 thì mô hình hoàn hảo, Gini = 0 thì mô hình dự đoán ngẫu nhiên

Tuy nhiên, chúng ta không nên để ROC, Gini = 1, gây ra overfit mô hình, nên cần phải cân bằng giữa precision và recall (tuỳ trường hợp lựa chọn criteria theo thực tế)

## Binomial test 
Kiểm tra xem mô hình có tốt hơn tỉ lệ cụ thể nào không, ta có hypothesis sau:
- H0: mô hình không tốt hơn tỉ lệ cụ thể (1 tail : p > p0) (2 tails : p != p0)
- H1: mô hình tốt hơn tỉ lệ cụ thể (1 tail : p < p0) (2 tails : p > p0)

## Hosmer-Lemeshow Test
Là kiểm định đánh giá mức độ phù hợp của mô hình logistic regression ( có thể áp dụng cho bài toán classification khác nếu có lớp cuối là logits, biến phụ thuộc nhị phân, phân nhóm dựa trên xác suất).
- H0: mô hình hiệu chỉnh tốt, không có sự khác biệt đáng kể giữa các xác suất quan sát và kì vọng
- H1: mô hình hiệu chỉnh không tốt, có sự khác biệt đáng kể giữa các xác suất quan sát và kì vọng 